In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

## Import Data

In [12]:
data = pd.DataFrame(pd.read_csv("prediction_result_3days_20220531_old.csv"))
data.iloc[:,-1] = data.iloc[:,-1].str.replace(',', '').astype(float)
data.columns = ['Ticker','Actual Price', 'Predicted Price', 'MSE', 'MAE', 'R2',
       'Market Share', 'marketcap']
data.head(2)

,Ticker,Actual Price,Predicted Price,MSE,MAE,R2,Market Share,marketcap
0,ABDN.L,175.403336,[163.5097],0.077741,0.278821,1,1899229952,333131268597.699219
1,BBOX.L,188.927795,[208.25749],0.279559,0.528733,1,1903740032,359669407279.820312


In [13]:
target_universe = pd.DataFrame(pd.read_excel("target_stock_universe.xlsx", sheet_name = '20220526'))

In [14]:
target_universe['Ticker'] = target_universe['Ticker'].str.split(' ').str[0]
target_universe['Ticker'] = target_universe['Ticker'] + ".L"

In [15]:
target_universe_list = target_universe.iloc[:,0].to_list()

## Selecting High Conviction Stocks through Volatility Measurement

In [16]:
from pandas_datareader import data as pdr
import datetime as dt
import yfinance as yf

yf.pdr_override()

stock_ticker = target_universe_list

start = dt.datetime(2017, 5, 25)
end = dt.datetime(2022, 5, 26)
stock_data_raw = pdr.get_data_yahoo(stock_ticker, start = start, end = end)

[*********************100%%**********************]  32 of 32 completed


In [17]:
stock_data = stock_data_raw['Adj Close']

In [18]:
stock_data = stock_data.reset_index()

### To model the volatility, we use EWMA, where the weights assigned decrease exponentially as we move back through time

In [19]:
start_date = '2017-05-25'
end_date = '2022-05-26'
lambda_value = 0.94

filtered_data = stock_data[(stock_data['Date'] >= start_date) & (stock_data['Date'] <= end_date)]

for stock in stock_ticker:
    filtered_data['DailyReturn_' + stock] = filtered_data[stock].pct_change()
    filtered_data['EWMA_' + stock] = filtered_data['DailyReturn_' + stock].ewm(span=int(1 / (1 - lambda_value))).mean()

In [20]:
volatility_data = []
for stock in stock_ticker:
    column_name = 'EWMA_' + stock
    stock_volatility = filtered_data[column_name].std()
    volatility_data.append({'Ticker': stock, 'Volatility': stock_volatility})

volatility_df = pd.DataFrame(volatility_data).sort_values(by='Volatility', ascending = False)
volatility_df

,Ticker,Volatility
19,TUI.L,0.009851
8,CCL.L,0.009533
16,EZJ.L,0.008905
27,FRAS.L,0.007473
26,WIZZ.L,0.007067
9,INVP.L,0.006806
7,ITV.L,0.006790
0,ICP.L,0.006779
10,CNA.L,0.006540
6,IDS.L,0.006420


In [21]:
start_date = '2022-05-19'
end_date = '2022-05-26'
lambda_value = 0.94

filtered_data = stock_data[(stock_data['Date'] >= start_date) & (stock_data['Date'] <= end_date)]

for stock in stock_ticker:
    filtered_data['DailyReturn_' + stock] = filtered_data[stock].pct_change()
    filtered_data['EWMA_' + stock] = filtered_data['DailyReturn_' + stock].ewm(span=int(1 / (1 - lambda_value))).mean()
    
volatility_data = []
for stock in stock_ticker:
    column_name = 'EWMA_' + stock
    stock_volatility = filtered_data[column_name].std()
    volatility_data.append({'Ticker': stock, 'Volatility': stock_volatility})

volatility_df = pd.DataFrame(volatility_data).sort_values(by='Volatility', ascending = False)
volatility_df

,Ticker,Volatility
6,IDS.L,0.024107
16,EZJ.L,0.017789
10,CNA.L,0.016683
7,ITV.L,0.016012
4,HIK.L,0.013573
27,FRAS.L,0.012098
9,INVP.L,0.011961
1,ABDN.L,0.011658
26,WIZZ.L,0.010716
23,EMG.L,0.010331
